# PROJECT02_GROUP12
Group members:  Krishna Sai Singamneni, Ajay Kumar Aluwala, Akshay Mirajkar, Venkata Pavani Pasupuleti, Charith Sarvareddy 

# PART I: Replicating Models from Class

# QUESTION 1

<hr style="border:2px solid black">
1. Consider the hard candy example from class. The associated data is in the file candy.csv. Develop the following models discussed in class using maximum likelihood estimation (MLE):  

(a) the Poisson model,  
(b) the NBD model,  
(c) the Zero Inflated NBD model, and   
(d) Finite Mixture models for 2, 3, and 4 segments.    

Report your code and all relevant details, including the estimated values of the parameters for each model and the corresponding log-likelihood values. Please add comments to your code to make it easy to understand.
<hr style="border:2px solid black">

In [1]:
import os 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.optimize import minimize
from math import e
from math import gamma
from scipy.special import factorial
from collections import Counter

In [2]:
os.chdir(r'D:\ACADS\SEMESTER I\BUAN 6383\PROJECTS\PROJECT 2')
df = pd.read_csv(r'candy.csv')
df

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44
5,5,25
6,6,26
7,7,15
8,8,15
9,9,10


<hr style="border:2px solid black">
(a) Poisson Model
<hr style="border:2px solid black">

In [3]:
# Define the Log Likelihood Function needed
def LLp(params,p,k):
    Lambda = params
    ll = []
    for i in range(len(p)):
        ll = np.append(ll,p[i]*((k[i]*np.log(Lambda))-(Lambda)-(np.log(gamma(k[i]+1)))))                
    return ll
            
# Negative of log likelihood for minimization function
def NLLp(params, p, k):
    return (-1)*(np.sum(LLp(params, p, k)))

# Defining p = 'No of people' and k = 'No of Packs'
p = np.array(df['People'])
k = np.array(df['Packs'])
            
# Maximising the log likelihood
soln_p = minimize(NLLp,
                  args = (p,k),
                  x0 = np.array((1)),
                  bounds = [(0.000001,None)],
                  tol=1e-10,
                  options={'ftol' : 1e-8},)

# Get the optimal solution
Lambda = soln_p.x[0]
ll_p = (-1)*(soln_p.fun)
print(f"Optimal Solution for Poisson Model:")
print(f"Lambda = {round(Lambda,4)}")
print(f"Log likelihood = {round(ll_p,2)}")

Optimal Solution for Poisson Model:
Lambda = 3.9912
Log likelihood = -1545.0


<hr style="border:2px solid black">
(b) NBD Model
<hr style="border:2px solid black">

In [4]:
# Define the Log Likelihood Function needed
def PROB(params,k,t):
    n,alpha = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t)))*PROB(params,k-1,t)

def LLnbd(params,t,p,K):
    ll = []
    for i in range(len(p)):
        ll = np.append(ll,(p[i]*np.log(PROB(params,K[i],t))))
    return ll

# Negative of log likelihood for minimization function
def NLLnbd(params,t,p,K):
    return (-1)*(np.sum(LLnbd(params,t,p,K)))
            
# Defining p = 'No of people' and K = 'No of Packs'
p = np.array(df['People'])
K = np.array(df['Packs'])
t = 1
            
# Maximising the log likelihood
soln_nbd = minimize(NLLnbd,
                    args = (t,p,K),
                    x0 = np.array((1,1)),
                    bounds = [(0.000001,None),(0.000001,None)],
                    tol=1e-10,
                    options={'ftol' : 1e-8},)

# Get the optimal solution
n = soln_nbd.x[0]
alpha = soln_nbd.x[1]
ll_nbd = (-1)*(soln_nbd.fun)
print(f"Optimal Solution for NBD Model:")
print(f"Shape Parameter, n = {round(n,6)}")
print(f"Scale Paramenter, alpha = {round(alpha,6)}")
print(f"Log likelihood = {round(ll_nbd,2)}")

Optimal Solution for NBD Model:
Shape Parameter, n = 0.997636
Scale Paramenter, alpha = 0.24996
Log likelihood = -1140.02


<hr style="border:2px solid black">
(c) Zero Inflated NBD
<hr style="border:2px solid black">

In [5]:
# Define the Log Likelihood Function needed
def PROB(alpha,n,k,t):
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t)))*PROB(alpha,n,k-1,t)

def LLzinbd(params,p,K):
    pie,alpha,n = params
    ll = p[0]*np.log(pie + ((1-pie)*PROB(alpha,n,K[0],1)))
    for i in range(1,len(p)):
        ll += p[i]*np.log((1-pie)*PROB(alpha,n,K[i],1))
    return ll

# Negative of log likelihood for minimization function
def NLLzinbd(params,p,K):
    return (-1)*((LLzinbd(params,p,K)))
            
# Defining p = 'No of people' and K = 'No of Packs'
p = np.array(df['People'])
K = np.array(df['Packs'])
            
# Maximising the log likelihood
soln_zinbd = minimize(NLLzinbd,
                      args = (p,K),
                      x0 = np.array((1,1,1)),
                      bounds = [(0.000001,0.999999),(0.000001,None),(0.000001,None)],
                      tol=1e-10,
                      options={'ftol' : 1e-8},)

# Get the optimal solution
pie = soln_zinbd.x[0]
n = soln_zinbd.x[2]
alpha = soln_zinbd.x[1]
ll_zinbd = (-1)*(soln_zinbd.fun)
print(f"Optimal Solution for Zero-Inflated NBD Model :")
print(f"Fraction, pie = {round(pie,6)}")
print(f"Shape Parameter, n = {round(n,6)}")
print(f"Scale Paramenter, alpha = {round(alpha,6)}")
print(f"Log likelihood = {round(ll_zinbd,2)}")

Optimal Solution for Zero-Inflated NBD Model :
Fraction, pie = 0.11309
Shape Parameter, n = 1.503988
Scale Paramenter, alpha = 0.334199
Log likelihood = -1136.17


<hr style="border:2px solid black">
(d) Finite Mixture Models
<hr style="border:2px solid black">

In [6]:
# Define the Log Likelihood Function needed
def LL2seg(params,p,k):
    pie,lambda1,lambda2 = params
    ll = []
    for i in range(len(p)):
        ll = np.append(ll,p[i]*(np.log((pie*(poisson.pmf(k[i],lambda1)))+((1-pie)*(poisson.pmf(k[i],lambda2))))))                
    return ll
            
# Negative of log likelihood for minimization function
def NLL2seg(params, p, k):
    return (-1)*(np.sum(LL2seg(params, p, k)))

# Defining p = 'No of people' and k = 'No of Packs'
p = np.array(df['People'])
k = np.array(df['Packs'])
            
# Maximising the log likelihood
soln_2seg = minimize(NLL2seg,
                     args = (p,k),
                     x0 = np.array((1,1,1)),
                     bounds = [(0.000001,0.999999),(0.000001,None),(0.000001,None)],
                     tol=1e-10,
                     options={'ftol' : 1e-8},)

# Get the optimal solution
pie_2seg = soln_2seg.x[0]
lambda1_2seg = soln_2seg.x[1]
lambda2_2seg = soln_2seg.x[2]
ll_2seg = (-1)*(soln_2seg.fun)
print(f"Optimal Solution for 2 Segment Finite Mixture Model:")
print(f"pie1 = {round(pie_2seg,4)}")
print(f"pie2 = {round(1-pie_2seg,4)}")
print(f"Lambda1 = {round(lambda1_2seg,4)}")
print(f"Lambda2 = {round(lambda2_2seg,4)}")
print(f"Log likelihood = {round(ll_2seg,2)}")

Optimal Solution for 2 Segment Finite Mixture Model:
pie1 = 0.2991
pie2 = 0.7009
Lambda1 = 9.1207
Lambda2 = 1.8022
Log likelihood = -1188.83


In [7]:
# Define the Log Likelihood Function needed
def LL3seg(params,theta3,p,k):
    theta1,theta2,lambda1,lambda2,lambda3 = params
    ll = []
    for i in range(len(p)):
        ll = np.append(ll,p[i]*(np.log((((e**(theta1))/((e**(theta1))+(e**(theta2))+(e**(theta3))))*(poisson.pmf(k[i],lambda1)))+(((e**(theta2))/((e**(theta1))+(e**(theta2))+(e**(theta3))))*(poisson.pmf(k[i],lambda2)))+(((e**(theta3))/((e**(theta1))+(e**(theta2))+(e**(theta3))))*(poisson.pmf(k[i],lambda3))))))                
    return ll
            
# Negative of log likelihood for minimization function
def NLL3seg(params,theta3,p,k):
    return (-1)*(np.sum(LL3seg(params,theta3,p,k)))

# Defining p = 'No of people' and k = 'No of Packs'
p = np.array(df['People'])
k = np.array(df['Packs'])
theta3 = 0
            
# Maximising the log likelihood
soln_3seg = minimize(NLL3seg,
                     args = (theta3,p,k),
                     x0 = np.array((1,2,1,1,1)),
                     bounds = [(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None)],
                     tol=1e-10,
                     options={'ftol' : 1e-8},)

# Get the optimal solution
pie1_3seg = e**(soln_3seg.x[0])/(e**(soln_3seg.x[0])+e**(soln_3seg.x[1])+1)
pie2_3seg = e**(soln_3seg.x[1])/(e**(soln_3seg.x[0])+e**(soln_3seg.x[1])+1)
lambda1_3seg = soln_3seg.x[2]
lambda2_3seg = soln_3seg.x[3]
lambda3_3seg = soln_3seg.x[4]
ll_3seg = (-1)*(soln_3seg.fun)
print(f"Optimal Solution for 3 Segment Finite Mixture Model:")
print(f"pie1 = {round(pie1_3seg,4)}")
print(f"pie2 = {round(pie2_3seg,4)}")
print(f"pie3 = {round(1-pie1_3seg-pie2_3seg,4)}")
print(f"Lambda1 = {round(lambda1_3seg,4)}")
print(f"Lambda2 = {round(lambda2_3seg,4)}")
print(f"Lambda3 = {round(lambda3_3seg,4)}")
print(f"Log likelihood = {round(ll_3seg,2)}")

Optimal Solution for 3 Segment Finite Mixture Model:
pie1 = 0.5433
pie2 = 0.18
pie3 = 0.2768
Lambda1 = 3.4833
Lambda2 = 11.2158
Lambda3 = 0.2906
Log likelihood = -1132.04


In [8]:
# Define the Log Likelihood Function needed
def LL4seg(params,theta4,p,k):
    theta1,theta2,theta3,lambda1,lambda2,lambda3,lambda4 = params
    ll = []
    for i in range(len(p)):
        ll = np.append(ll,p[i]*(np.log((((e**(theta1))/((e**(theta1))+(e**(theta2))+(e**(theta3))+(e**(theta4))))*(poisson.pmf(k[i],lambda1)))+(((e**(theta2))/((e**(theta1))+(e**(theta2))+(e**(theta3))+(e**(theta4))))*(poisson.pmf(k[i],lambda2)))+(((e**(theta3))/((e**(theta1))+(e**(theta2))+(e**(theta3))+(e**(theta4))))*(poisson.pmf(k[i],lambda3)))+(((e**(theta4))/((e**(theta1))+(e**(theta2))+(e**(theta3))+(e**(theta4))))*(poisson.pmf(k[i],lambda4))))))                
    return ll
            
# Negative of log likelihood for minimization function
def NLL4seg(params,theta4,p,k):
    return (-1)*(np.sum(LL4seg(params,theta4,p,k)))

# Defining p = 'No of people' and k = 'No of Packs'
p = np.array(df['People'])
k = np.array(df['Packs'])
theta4 = 0
            
# Maximising the log likelihood
soln_4seg = minimize(NLL4seg,
                     args = (theta4,p,k),
                     x0 = np.array((1,2,3,1,1,1,1)),
                     bounds = [(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None),(0.000001,None)],
                     tol=1e-10,
                     options={'ftol' : 1e-8},)

# Get the optimal solution
pie1_4seg = e**(soln_4seg.x[0])/(e**(soln_4seg.x[0])+e**(soln_4seg.x[1])+e**(soln_4seg.x[2])+1)
pie2_4seg = e**(soln_4seg.x[1])/(e**(soln_4seg.x[0])+e**(soln_4seg.x[1])+e**(soln_4seg.x[2])+1)
pie3_4seg = e**(soln_4seg.x[2])/(e**(soln_4seg.x[0])+e**(soln_4seg.x[1])+e**(soln_4seg.x[2])+1)
lambda1_4seg = soln_4seg.x[3]
lambda2_4seg = soln_4seg.x[4]
lambda3_4seg = soln_4seg.x[5]
lambda4_4seg = soln_4seg.x[6]
ll_4seg = (-1)*(soln_4seg.fun)
print(f"Optimal Solution for 4 Segment Finite Mixture Model:")
print(f"pie1 = {round(pie1_4seg,4)}")
print(f"pie2 = {round(pie2_4seg,4)}")
print(f"pie3 = {round(pie3_4seg,4)}")
print(f"pie4 = {round(1-pie1_4seg-pie2_4seg-pie3_4seg,4)}")
print(f"Lambda1 = {round(lambda1_4seg,4)}")
print(f"Lambda2 = {round(lambda2_4seg,4)}")
print(f"Lambda3 = {round(lambda3_4seg,4)}")
print(f"Lambda4 = {round(lambda4_4seg,4)}")
print(f"Log likelihood = {round(ll_4seg,2)}")

Optimal Solution for 4 Segment Finite Mixture Model:
pie1 = 0.1514
pie2 = 0.2442
pie3 = 0.1017
pie4 = 0.5027
Lambda1 = 7.4187
Lambda2 = 0.2048
Lambda3 = 12.8731
Lambda4 = 3.0021
Log likelihood = -1130.07


<hr style="border:2px solid black">
2. Evaluate the models developed; explain which of them is best, and why. Are there any significant differences among the results from these models? If so, what exactly are these differences? Discuss what you believe could be causing the differences.
<hr style="border:2px solid black">

<hr style="border:2px solid black">
BIC
<hr style="border:2px solid black">

In [9]:
# BIC: k*ln(n)-2*LL
BIC_p = (1*np.log(np.sum(df['People'])))-(2*ll_p)
BIC_nbd = (2*np.log(np.sum(df['People'])))-(2*ll_nbd)
BIC_zinbd = (3*np.log(np.sum(df['People'])))-(2*ll_zinbd)
BIC_2seg = (3*np.log(np.sum(df['People'])))-(2*ll_2seg)
BIC_3seg = (5*np.log(np.sum(df['People'])))-(2*ll_3seg)
BIC_4seg = (7*np.log(np.sum(df['People'])))-(2*ll_4seg)


# Printing the BIC values for all the models
print(f"BIC for Poisson model is {round(BIC_p,2)}")
print(f"BIC for NBD model is {round(BIC_nbd,2)}")
print(f"BIC for Zero-Inflated NBD model is {round(BIC_zinbd,2)}")
print(f"BIC for 2 Segment Finite Mixture model is {round(BIC_2seg,2)}")
print(f"BIC for 3 Segment Finite Mixture model is {round(BIC_3seg,2)}")
print(f"BIC for 4 Segment Finite Mixture model is {round(BIC_4seg,2)}")

BIC for Poisson model is 3096.12
BIC for NBD model is 2292.29
BIC for Zero-Inflated NBD model is 2290.7
BIC for 2 Segment Finite Mixture model is 2396.03
BIC for 3 Segment Finite Mixture model is 2294.7
BIC for 4 Segment Finite Mixture model is 2303.0


<hr style="border:2px solid black">
AIC
<hr style="border:2px solid black">

In [10]:
# AIC: 2k-2*LL
AIC_p = (2*1)-(2*ll_p)
AIC_nbd = (2*2)-(2*ll_nbd)
AIC_zinbd = (2*3)-(2*ll_zinbd)
AIC_2seg = (2*3)-(2*ll_2seg)
AIC_3seg = (2*5)-(2*ll_3seg)
AIC_4seg = (2*7)-(2*ll_4seg)

# Printing the AIC values for all the models
print(f"AIC for Poisson model is {round(AIC_p,2)}")
print(f"AIC for NBD model is {round(AIC_nbd,2)}")
print(f"AIC for Zero-Inflated NBD model is {round(AIC_zinbd,2)}")
print(f"AIC for 2 Segment Finite Mixture model is {round(AIC_2seg,2)}")
print(f"AIC for 3 Segment Finite Mixture model is {round(AIC_3seg,2)}")
print(f"AIC for 4 Segment Finite Mixture model is {round(AIC_4seg,2)}")

AIC for Poisson model is 3091.99
AIC for NBD model is 2284.05
AIC for Zero-Inflated NBD model is 2278.33
AIC for 2 Segment Finite Mixture model is 2383.67
AIC for 3 Segment Finite Mixture model is 2274.09
AIC for 4 Segment Finite Mixture model is 2274.14


<hr style="border:2px solid black">
3. Based on the 2, 3, and 4-segment finite mixture models, how many packs are the following customers likely to purchase over the next 8 weeks?

(a) a customer who purchased 5 packs in the past week, and   
(b) a customer who purchased 9 packs in the past week.
<hr style="border:2px solid black">

<hr style="border:2px solid black">
(a) a customer who purchased 5 packs in the past week
<hr style="border:2px solid black">

<hr style="border:2px solid black">
(i) 2 Segment Finite Mixture model
<hr style="border:2px solid black">

In [11]:
P2_seg1 = ((pie_2seg)*(poisson.pmf(5,lambda1_2seg)))/(((pie_2seg)*(poisson.pmf(5,lambda1_2seg)))+((1-pie_2seg)*(poisson.pmf(5,lambda2_2seg))))
P2_seg2 = ((1-pie_2seg)*(poisson.pmf(5,lambda2_2seg)))/(((pie_2seg)*(poisson.pmf(5,lambda1_2seg)))+((1-pie_2seg)*(poisson.pmf(5,lambda2_2seg))))

Exp_2seg = 8*((lambda1_2seg*P2_seg1)+(lambda2_2seg*P2_seg2))

print(f"For the 2 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P2_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P2_seg2,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_2seg,2)}")

For the 2 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.4845
Probability of customer being in segment 2 is 0.5155
Expected number of purchases in 8 weeks is 42.78


In [12]:
# If classifying customers into segment 2
Exp_2seg_ = 8*(lambda2_2seg)

print(f"If classifying customers into segment 2:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_2seg_,2)}")

If classifying customers into segment 2:
Expected number of purchases in 8 weeks is 14.42


<hr style="border:2px solid black">
(ii) 3 Segment Finite Mixture model
<hr style="border:2px solid black">

In [13]:
P3_seg1 = ((pie1_3seg)*(poisson.pmf(5,lambda1_3seg)))/(((pie1_3seg)*(poisson.pmf(5,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(5,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(5,lambda3_3seg))))
P3_seg2 = ((pie2_3seg)*(poisson.pmf(5,lambda2_3seg)))/(((pie1_3seg)*(poisson.pmf(5,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(5,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(5,lambda3_3seg))))
P3_seg3 = ((1-pie1_3seg-pie2_3seg)*(poisson.pmf(5,lambda3_3seg)))/(((pie1_3seg)*(poisson.pmf(5,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(5,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(5,lambda3_3seg))))

Exp_3seg = 8*((lambda1_3seg*P3_seg1)+(lambda2_3seg*P3_seg2)+(lambda3_3seg*P3_seg3))

print(f"For the 3 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P3_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P3_seg2,4)}")
print(f"Probability of customer being in segment 3 is {round(P3_seg3,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_3seg,2)}")

For the 3 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.9521
Probability of customer being in segment 2 is 0.0478
Probability of customer being in segment 3 is 0.0
Expected number of purchases in 8 weeks is 30.83


In [14]:
# If classifying customers into segment 1
Exp_3seg_ = 8*(lambda1_3seg)

print(f"If classifying customers into segment 1:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_3seg_,2)}")

If classifying customers into segment 1:
Expected number of purchases in 8 weeks is 27.87


<hr style="border:2px solid black">
(iii) 4 Segment Finite Mixture model
<hr style="border:2px solid black">

In [15]:
P4_seg1 = ((pie1_4seg)*(poisson.pmf(5,lambda1_4seg)))/(((pie1_4seg)*(poisson.pmf(5,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(5,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(5,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(5,lambda4_4seg))))
P4_seg2 = ((pie2_4seg)*(poisson.pmf(5,lambda2_4seg)))/(((pie1_4seg)*(poisson.pmf(5,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(5,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(5,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(5,lambda4_4seg))))
P4_seg3 = ((pie3_4seg)*(poisson.pmf(5,lambda3_4seg)))/(((pie1_4seg)*(poisson.pmf(5,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(5,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(5,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(5,lambda4_4seg))))
P4_seg4 = ((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(5,lambda4_4seg)))/(((pie1_4seg)*(poisson.pmf(5,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(5,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(5,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(5,lambda4_4seg))))

Exp_4seg = 8*((lambda1_4seg*P4_seg1)+(lambda2_4seg*P4_seg2)+(lambda3_4seg*P4_seg3)+(lambda4_4seg*P4_seg4))

print(f"For the 4 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P4_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P4_seg2,4)}")
print(f"Probability of customer being in segment 3 is {round(P4_seg3,4)}")
print(f"Probability of customer being in segment 4 is {round(P4_seg4,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_4seg,2)}")

For the 4 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.2482
Probability of customer being in segment 2 is 0.0
Probability of customer being in segment 3 is 0.0112
Probability of customer being in segment 4 is 0.7406
Expected number of purchases in 8 weeks is 33.67


In [16]:
# If classifying customers into segment 4
Exp_4seg_ = 8*(lambda4_4seg)

print(f"If classifying customers into segment 4:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_4seg_,2)}")

If classifying customers into segment 4:
Expected number of purchases in 8 weeks is 24.02


<hr style="border:2px solid black">
(b) a customer who purchased 9 packs in the past week
<hr style="border:2px solid black">

<hr style="border:2px solid black">
(i) 2 Segment Finite Mixture model
<hr style="border:2px solid black">

In [17]:
P2_seg1 = ((pie_2seg)*(poisson.pmf(9,lambda1_2seg)))/(((pie_2seg)*(poisson.pmf(9,lambda1_2seg)))+((1-pie_2seg)*(poisson.pmf(9,lambda2_2seg))))
P2_seg2 = ((1-pie_2seg)*(poisson.pmf(9,lambda2_2seg)))/(((pie_2seg)*(poisson.pmf(9,lambda1_2seg)))+((1-pie_2seg)*(poisson.pmf(9,lambda2_2seg))))

Exp_2seg = 8*((lambda1_2seg*P2_seg1)+(lambda2_2seg*P2_seg2))

print(f"For the 2 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P2_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P2_seg2,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_2seg,2)}")

For the 2 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.9984
Probability of customer being in segment 2 is 0.0016
Expected number of purchases in 8 weeks is 72.87


In [18]:
# If classifying customers into segment 1
Exp_2seg_ = 8*(lambda1_2seg)

print(f"If classifying customers into segment 1:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_2seg_,2)}")

If classifying customers into segment 1:
Expected number of purchases in 8 weeks is 72.97


<hr style="border:2px solid black">
(ii) 3 Segment Finite Mixture model
<hr style="border:2px solid black">

In [19]:
P3_seg1 = ((pie1_3seg)*(poisson.pmf(9,lambda1_3seg)))/(((pie1_3seg)*(poisson.pmf(9,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(9,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(9,lambda3_3seg))))
P3_seg2 = ((pie2_3seg)*(poisson.pmf(9,lambda2_3seg)))/(((pie1_3seg)*(poisson.pmf(9,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(9,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(9,lambda3_3seg))))
P3_seg3 = ((1-pie1_3seg-pie2_3seg)*(poisson.pmf(9,lambda3_3seg)))/(((pie1_3seg)*(poisson.pmf(9,lambda1_3seg)))+((pie2_3seg)*(poisson.pmf(9,lambda2_3seg)))+((1-pie1_3seg-pie2_3seg)*(poisson.pmf(9,lambda3_3seg))))

Exp_3seg = 8*((lambda1_3seg*P3_seg1)+(lambda2_3seg*P3_seg2)+(lambda3_3seg*P3_seg3))

print(f"For the 3 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P3_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P3_seg2,4)}")
print(f"Probability of customer being in segment 3 is {round(P3_seg3,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_3seg,2)}")

For the 3 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.1562
Probability of customer being in segment 2 is 0.8438
Probability of customer being in segment 3 is 0.0
Expected number of purchases in 8 weeks is 80.06


In [20]:
# If classifying customers into segment 2
Exp_3seg_ = 8*(lambda2_3seg)

print(f"If classifying customers into segment 2:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_3seg_,2)}")

If classifying customers into segment 2:
Expected number of purchases in 8 weeks is 89.73


<hr style="border:2px solid black">
(iii) 4 Segment Finite Mixture model
<hr style="border:2px solid black">

In [21]:
P4_seg1 = ((pie1_4seg)*(poisson.pmf(9,lambda1_4seg)))/(((pie1_4seg)*(poisson.pmf(9,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(9,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(9,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(9,lambda4_4seg))))
P4_seg2 = ((pie2_4seg)*(poisson.pmf(9,lambda2_4seg)))/(((pie1_4seg)*(poisson.pmf(9,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(9,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(9,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(9,lambda4_4seg))))
P4_seg3 = ((pie3_4seg)*(poisson.pmf(9,lambda3_4seg)))/(((pie1_4seg)*(poisson.pmf(9,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(9,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(9,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(9,lambda4_4seg))))
P4_seg4 = ((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(9,lambda4_4seg)))/(((pie1_4seg)*(poisson.pmf(9,lambda1_4seg)))+((pie2_4seg)*(poisson.pmf(9,lambda2_4seg)))+((pie3_4seg)*(poisson.pmf(9,lambda3_4seg)))+((1-pie1_4seg-pie2_4seg-pie3_4seg)*(poisson.pmf(9,lambda4_4seg))))

Exp_4seg = 8*((lambda1_4seg*P4_seg1)+(lambda2_4seg*P4_seg2)+(lambda3_4seg*P4_seg3)+(lambda4_4seg*P4_seg4))

print(f"For the 4 Segment Finite Mixture model:")
print(f"Probability of customer being in segment 1 is {round(P4_seg1,4)}")
print(f"Probability of customer being in segment 2 is {round(P4_seg2,4)}")
print(f"Probability of customer being in segment 3 is {round(P4_seg3,4)}")
print(f"Probability of customer being in segment 4 is {round(P4_seg4,4)}")
print(f"Expected number of purchases in 8 weeks is {round(Exp_4seg,2)}")

For the 4 Segment Finite Mixture model:
Probability of customer being in segment 1 is 0.6712
Probability of customer being in segment 2 is 0.0
Probability of customer being in segment 3 is 0.2751
Probability of customer being in segment 4 is 0.0537
Expected number of purchases in 8 weeks is 69.45


In [22]:
# If classifying customers into segment 1
Exp_4seg_ = 8*(lambda1_4seg)

print(f"If classifying customers into segment 1:")
print(f"Expected number of purchases in 8 weeks is {round(Exp_4seg_,2)}")

If classifying customers into segment 1:
Expected number of purchases in 8 weeks is 59.35


# PART II: Analysis of New Data

<hr style="border:2px solid black">
articles.csv contains the number of publications by 915 doctoral candidates (articles), along with five predictors:

1. female: 1 if candidate was female, 0 otherwise 

2. married: 1 if candidate was married, 0 otherwise

3. kids: number of children aged <= 5

4. prestige: prestige of the candidate's department (higher is better)

5. mentorpubs: number of publications by the candidate's mentor over the past 3 years

Your task is to to predict the number of articles as a function of the five independent variables.
<hr style="border:2px solid black">

In [23]:
os.chdir(r'D:\ACADS\SEMESTER I\BUAN 6383\PROJECTS\PROJECT 2')
df1 = pd.read_csv(r'articles.csv')
df1

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


In [24]:
df1.isnull().sum()

articles    0
female      0
married     0
kids        0
prestige    0
menpubs     0
dtype: int64

# QUESTION 1 

<hr style="border:2px solid black">
1. Estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the managerial takeaways | which customer characteristics seem to be important?
<hr style="border:2px solid black">

In [25]:
# Define the Log Likelihood Function needed
# We divide the log likelihood into 3 parts and calculate the total sum
def PART1(params,l,k):
    #beta1,beta2,beta3,beta4,beta5,lambda0 = params
    sigma1 = 0
    for i in range(len(k)):
        reg = 0
        for j in range(len(l[i])):
            reg += params[j]*l[i][j]
        reg = np.log((e**(reg))*(params[j+1]))
        sigma1 += k[i]*reg
    return sigma1

def PART2(params,l):
    #beta1,beta2,beta3,beta4,beta5 = params
    sigma2 = 0
    for i in range(len(l)):
        reg = 0
        for j in range(len(l[i])):
            reg += params[j]*l[i][j]
        sigma2 += e**(reg)
    return params[len(params)-1]*sigma2

def PART3(k):
    sigma3 = 0
    for i in k:
        sigma3 += np.log(float(factorial(i)))
    return sigma3

def LLpr(params,l,k):
    return -1*(PART1(params,l,k)-PART2(params,l)-PART3(k))

# Defining parameters and maximising log likelihood
l = np.array(df1[['female','married','kids','prestige','menpubs']])
k = np.array(df1['articles'])
soln_pr = minimize(LLpr,
                   args = (l,k),
                   x0 = np.array((0.1,0.1,0.1,0.1,0.1,1)),
                   bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None)],
                   tol=1e-10,
                   options={'ftol' : 1e-8})

# Get the optimal solution
beta1 = soln_pr.x[0]
beta2 = soln_pr.x[1]
beta3 = soln_pr.x[2]
beta4 = soln_pr.x[3]
beta5 = soln_pr.x[4]
lambda0 = soln_pr.x[5]
ll_pr = (-1)*soln_pr.fun
print(f"Optimal Solution for Poisson Regression Model:")
print(f"lambda0 = {round(lambda0,4)}")
print(f"beta1 = {round(beta1,4)}")
print(f"beta2 = {round(beta2,4)}")
print(f"beta3 = {round(beta3,4)}")
print(f"beta4 = {round(beta4,4)}")
print(f"beta5 = {round(beta5,4)}")
print(f"Log likelihood = {round(ll_pr,2)}")       

Optimal Solution for Poisson Regression Model:
lambda0 = 1.3568
beta1 = -0.2246
beta2 = 0.1551
beta3 = -0.185
beta4 = 0.0127
beta5 = 0.0255
Log likelihood = -1651.06


# QUESTION 2

<hr style="border:2px solid black">
2. Estimate all relevant parameters for NBD Regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the managerial takeaways | which customer characteristics seem to be important?
<hr style="border:2px solid black">

In [26]:
# Define the Log Likelihood Function needed
# As we are unable to proper optimization with a single function due to log,
# we divide the log likelihood into 2 parts and calculate the total sum

def PART1(params,l,k):
    n = params[len(l)+1]
    log_num = np.log(gamma(n+k))
    log_denom1 = np.log(gamma(n))
    log_denom2 = np.log(gamma(k+1))
    return log_num - (log_denom1 + log_denom2)

def PART2(params,l,k):
    alpha = params[len(l)]
    n = params[len(l)+1]
    reg = 0
    for j in range(len(l)):
        reg += params[j]*l[j]
    exp = e**reg
    log1 = n*(np.log(alpha) - np.log(alpha+exp))
    log2 = k*(np.log(exp) - np.log(alpha+exp))
    return log1 + log2

def LLnbdr(params,l,k):
    ll = []
    for i in range(len(k)):
        ll = np.append(ll,(PART1(params,l[i],int(k[i])) + PART2(params,l[i],int(k[i]))))
    return ll

# Negative of log likelihood for minimization function
def NLLnbdr(params,l,k):
    return (-1)*np.sum(LLnbdr(params,l,k))

# Defining parameters and maximising log likelihood
l = np.array(df1[['female','married','kids','prestige','menpubs']])
k = np.array(df1['articles'])

soln_nbdr = minimize(NLLnbdr,
                     args = (l,k),
                     x0 = np.array((0.1,0.1,0.1,0.1,0.1,1,1)),
                     bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None)],
                     tol=1e-10,
                     options={'ftol' : 1e-8})

# Get the optimal solution
beta1 = soln_nbdr.x[0]
beta2 = soln_nbdr.x[1]
beta3 = soln_nbdr.x[2]
beta4 = soln_nbdr.x[3]
beta5 = soln_nbdr.x[4]
alpha = soln_nbdr.x[5]
n = soln_nbdr.x[6]
ll_nbdr = (-1)*soln_nbdr.fun

print(f"Optimal Solution for NBD Regression Model:")
print(f"n = {round(n,4)}")
print(f"alpha = {round(alpha,4)}")
print(f"beta1 = {round(beta1,4)}")
print(f"beta2 = {round(beta2,4)}")
print(f"beta3 = {round(beta3,4)}")
print(f"beta4 = {round(beta4,4)}")
print(f"beta5 = {round(beta5,4)}")
print(f"Log likelihood = {round(ll_nbdr,6)}")

Optimal Solution for NBD Regression Model:
n = 2.2646
alpha = 1.7534
beta1 = -0.2163
beta2 = 0.1505
beta3 = -0.1763
beta4 = 0.0153
beta5 = 0.0291
Log likelihood = -1560.958347


# QUESTION 3

<hr style="border:2px solid black">
3. In this question, you will apply the ideas learned in this course to build a model that you have not seen before {The Zero Inflated NBD Regression. 

First, recall that zero inflated models view 0s as coming from 2 sources - (i) from a fraction pie who is 0 "by type" (in the context of this problem, these are candidates who will never publish), and (ii) from the remaining fraction (1-pie) who are likely to eventually become nonzero (these are candidates who will publish at some point, but have not done so yet). You can assume that the candidates in the latter group are distributed as a negative binomial (making the NBD regression appropriate for them). Explain the logic used in developing the model in detail. (hint: you do not need anything beyond what you have learned in the class to do this.)

Report your code, the estimated parameters and the maximum value of the log-likelihood. What are the
managerial takeaways | which customer characteristics seem to be important?
<hr style="border:2px solid black">

In [27]:
def PART1(params,l,k):
    n = params[len(l)+1]
    num = gamma(n+k)
    denom = gamma(n)*(factorial(k))
    return num/denom

def PART2(params,l,k):
    alpha = params[len(l)]
    n = params[len(l)+1]
    reg = 0
    for j in range(len(l)):
        reg += params[j]*l[j]
    exp = e**reg
    final = ((alpha/(alpha+exp))**n)*((exp/(alpha+exp))**k)
    return final

def LLzinbdr(params,l,k):
    pie = params[len(params)-1]
    ll = []
    for i in range(len(k)):
        if k[i] == 0:
            ll_zi = np.log(pie + ((1-pie)*(PART1(params,l[i],k[i]))*(PART2(params,l[i],k[i]))))
        else:
            ll_zi = np.log((1-pie)*(PART1(params,l[i],k[i]))*(PART2(params,l[i],k[i])))
        ll.append(ll_zi)
    return ll

# Negative of log likelihood for minimization function
def NLLzinbdr(params,l,k):
    return (-1)*np.sum(LLzinbdr(params,l,k))

# Defining parameters and maximising log likelihood
l = np.array(df1[['female','married','kids','prestige','menpubs']])
k = np.array(df1['articles'])

soln_zinbdr = minimize(NLLzinbdr,
                       args = (l,k),
                       x0 = np.array((0.1,0.1,0.1,0.1,0.1,1,1,0.5)),
                       bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,0.999999)],
                       tol=1e-10,
                       options={'ftol' : 1e-8})

# Get the optimal solution
beta1 = soln_zinbdr.x[0]
beta2 = soln_zinbdr.x[1]
beta3 = soln_zinbdr.x[2]
beta4 = soln_zinbdr.x[3]
beta5 = soln_zinbdr.x[4]
alpha = soln_zinbdr.x[5]
n = soln_zinbdr.x[6]
pie = soln_zinbdr.x[7]
ll_zinbdr = (-1)*soln_zinbdr.fun

print(f"Optimal Solution for Zero-Inflated NBD Regression Model:")
print(f"pie = {round(pie,6)}")
print(f"n = {round(n,4)}")
print(f"alpha = {round(alpha,4)}")
print(f"beta1 = {round(beta1,4)}")
print(f"beta2 = {round(beta2,4)}")
print(f"beta3 = {round(beta3,4)}")
print(f"beta4 = {round(beta4,4)}")
print(f"beta5 = {round(beta5,4)}")
print(f"Log likelihood = {round(ll_zinbdr,6)}")

Optimal Solution for Zero-Inflated NBD Regression Model:
pie = 1e-06
n = 2.2645
alpha = 1.7527
beta1 = -0.2165
beta2 = 0.1504
beta3 = -0.1767
beta4 = 0.0153
beta5 = 0.0291
Log likelihood = -1560.958404


# QUESTION 4

<hr style="border:2px solid black">
4. Evaluate the models developed; explain which of them is best, and why. Are there any significant differences among the results from these models? If so, what exactly are these differences? Discuss what you believe could be causing the diferences.
<hr style="border:2px solid black">

<hr style="border:2px solid black">
BIC
<hr style="border:2px solid black">

In [28]:
# BIC: k*ln(n)-2*LL
BIC_pr = (6*np.log(len(df1['articles'])))-(2*ll_pr)
BIC_nbdr = (7*np.log(len(df1['articles'])))-(2*ll_nbdr)
BIC_zinbdr = (8*np.log(len(df1['articles'])))-(2*ll_zinbdr)

# Printing the BIC values for all the models
print(f"BIC for Poisson Regression model is {round(BIC_pr,2)}")
print(f"BIC for NBD Regression model is {round(BIC_nbdr,2)}")
print(f"BIC for Zero Inflated NBD Regression model is {round(BIC_zinbdr,2)}")

BIC for Poisson Regression model is 3343.03
BIC for NBD Regression model is 3169.65
BIC for Zero Inflated NBD Regression model is 3176.47


<hr style="border:2px solid black">
AIC
<hr style="border:2px solid black">

In [29]:
# AIC: 2k-2*LL
AIC_pr = (6*2)-(2*ll_pr)
AIC_nbdr = (7*2)-(2*ll_nbdr)
AIC_zinbdr = (8*2)-(2*ll_zinbdr)

# Printing the AIC values for all the models
print(f"AIC for Poisson Regression model is {round(AIC_pr,2)}")
print(f"AIC for NBD Regression model is {round(AIC_nbdr,2)}")
print(f"AIC for Zero Inflated NBD Regression model is {round(AIC_zinbdr,2)}")

AIC for Poisson Regression model is 3314.11
AIC for NBD Regression model is 3135.92
AIC for Zero Inflated NBD Regression model is 3137.92
